In [2]:
import pandas as pd
import openpyxl
import warnings
import os

warnings.filterwarnings("ignore")

In [42]:
base_path = "BASE.xlsx"
resultado = "resultado.xlsx"

df_base = pd.read_excel(base_path)
df_base = df_base.iloc[:, 3:] # Eliminar columnas iniciales
df_base = df_base.iloc[7:, :] # Eliminar filas iniciales
df_base = df_base.rename(columns=df_base.iloc[0]) # Renombra las columnas
df_base = df_base[1:]
df_base = df_base.reset_index(drop=True) # Reinicia los indices
df_base = df_base.dropna(subset=["Cuenta","ACC"]) # Elimina filas con valores NaN
df_base = df_base.reset_index(drop=True)
columnas_deseadas = ["ACC", "Cuenta", "Demora", "     Importe en ML"]
df_base = df_base[columnas_deseadas]
df_base = df_base.rename(columns={"     Importe en ML": "Importe"})
df_base["Demora"] = df_base["Demora"].astype("Int64")
df_base["Importe"] = df_base["Importe"].astype(float)
# Condition 1
df_base["Status"] = df_base["Importe"].apply(lambda x: "DEUDA" if x > 0 else "SALDOS A FAVOR")
# Condition 2
df_base["Tipo Deuda"] = df_base["Demora"].apply(lambda x: "CORRIENTE" if x <= 0 else "VENCIDA")
# Condition 3
df_base["Saldo Final"] = df_base.apply(lambda row: row["Importe"] if (row["Status"] == "DEUDA" and row["Tipo Deuda"] == "VENCIDA") else (row["Importe"] if row["Status"] == "SALDOS A FAVOR" else "NO"), axis=1)
df_base = df_base[df_base["Saldo Final"] != "NO"]
df_base = df_base.sort_values(by=["Cuenta"], ascending=[True])
df_base = df_base.sort_values(by=["ACC"], ascending=[True])
df_base = df_base.sort_values(by=["Demora"], ascending=[False])
df_base = df_base.reset_index(drop=True)

ultima_fila = df_base.shape[0]
for i in range(ultima_fila):
    if df_base.loc[i, "Status"] == "DEUDA":
        saldoDeuda = df_base.loc[i, "Saldo Final"]
        for j in range(ultima_fila):
            if (
                df_base.loc[i, "Cuenta"]    == df_base.loc[j, "Cuenta"] and 
                df_base.loc[i, "ACC"]       == df_base.loc[j, "ACC"]    and 
                df_base.loc[j, "Status"]    == "SALDOS A FAVOR"
                ):
                saldoFavor = df_base.loc[j, "Saldo Final"]
                montoCompensar = min(saldoDeuda, abs(saldoFavor))
                df_base.loc[i, "Saldo Final"] = saldoDeuda - montoCompensar
                df_base.loc[j, "Saldo Final"] = saldoFavor + montoCompensar
                saldoDeuda = df_base.loc[i, "Saldo Final"]

df_base = df_base[(df_base["Tipo Deuda"] == "VENCIDA") & (df_base["Status"] == "DEUDA")]
df_base = df_base.reset_index(drop=True)

grouped_df = df_base.groupby(["Cuenta", "ACC"]).agg({"Demora": "max", "Saldo Final": "sum"})

df_final = grouped_df.reset_index()[["Cuenta", "ACC", "Saldo Final", "Demora"]]
df_final = df_final.rename(columns={"Cuenta":"Cod Cliente", "ACC":"Área Ctrl", "Saldo Final":"Deuda Vencida", "Demora":"Días de Morosidad"})

df_final.to_excel(resultado, index=False)
os.startfile(resultado)

print(df_final.shape)
df_final.head()

(80, 4)


,Cod Cliente,Área Ctrl,Deuda Vencida,Días de Morosidad
0,4070150,PE01,1051.72,410
1,4070150,PE16,215889.65,1146
2,4070209,PE10,693.84,1
3,4070435,PE02,230.02,6
4,4070587,PE01,1525.5,2
